## Automação de Indicadores

### Objetivo: Treinar e criar um projeto completo que envolva a automatização de um processo feito no computador


In [22]:
#Importações
import pandas as pd 
from pathlib import Path
import win32com.client as win32

In [23]:
#Lendo os dados das bases de dados

vendas_df = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
emails_df = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas_df =  pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')

In [24]:
#Incluir o nome da loja em vendas
vendas = vendas_df.merge(lojas_df, on='ID Loja')
#display(vendas)

In [25]:
#Criando uma tabela para cada uma das lojas
dicionario_lojas = {}

for loja in lojas_df['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :]

In [26]:
#Pegando o dia do indicador - Minha maior data
dia_indicador = vendas['Data'].max()
print(dia_indicador)

2019-12-26 00:00:00


In [27]:
#Criando a pasta de backup para salvar os arquivos

caminho_backup = Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()

lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]


#Criando as pastas na pasta de backup

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir(exist_ok=True)

    nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
    local_arquivo = caminho_backup / loja / nome_arquivo

    dicionario_lojas[loja].to_excel(local_arquivo)

#print(lista_nomes_backup)

In [41]:
# Metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtd_produtos_dia = 4
meta_qtd_produtos_ano = 120
meta_ticket_medio_dia = 500
meta_ticket_medio_ano = 500

In [ ]:
#Calcular Indicadores das Lojas
for loja in dicionario_lojas:

    #Tabela auxiliar 
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data'] == dia_indicador,:]

    #Faturamento -> Soma da coluna Valor final
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia =  vendas_loja_dia['Valor Final'].sum()

    #print('Faturamento Anual: {:,.2f}'.format(faturamento_ano))
    #print('Faturamento do Dia: {:,.2f}'.format(faturamento_dia))

    #Diversidade de produtos
    quantidade_produtos_ano = len(vendas_loja['Produto'].unique())
    quantidade_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #print('Quantidade produtos vendido anual: {:,.2f}'.format(quantidade_produtos_ano))
    #print('Quantidade produtos vendido dia: {:,.2f}'.format(quantidade_produtos_dia))


    #ticket médio ano
    valor_venda =  vendas_loja.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    
    #print(ticket_medio_ano)

    #ticket_medio_dia
    valor_venda_dia =  vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    #print(ticket_medio_dia)

    #Envio de e-mail
    outlook = win32.Dispatch('outlook.application')

    nome_gerente = emails_df.loc[emails_df['Loja'] == loja ,'Gerente'].values[0]

    mail = outlook.CreateItem(0)
    mail.To = emails_df.loc[emails_df['Loja'] == loja ,'E-mail'].values[0]
    mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'


    #Variáveis para adicionar cor ao icon 

    cor_faturamento_dia = 'green' if faturamento_dia >= meta_faturamento_dia else 'red'
    cor_quantidade_dia = 'green' if quantidade_produtos_dia >= meta_qtd_produtos_dia else 'red'
    cor_ticked_dia = '' 'green' if ticket_medio_dia >= meta_ticket_medio_dia else 'red'

    cor_faturamento_ano = 'green' if faturamento_ano >= meta_faturamento_ano else 'red'
    cor_quantidade_ano = 'green' if quantidade_produtos_ano >= meta_qtd_produtos_ano else 'red'
    cor_ticked_ano = '' 'green' if ticket_medio_ano >= ticket_medio_ano else 'red'


    mail.HTMLBody = f"""
    <p>Bom dia {nome_gerente},</p>

    <p>O resultado da venda de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja} </strong> foi de: {faturamento_dia}</p>


    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cénario Dia</th>
    </tr>

    <tr>
        <td>Faturamento</td>
        <td style='text-align: center;'>R$ {faturamento_dia:.2f}</td>
        <td style='text-align: center;'>R$ {meta_faturamento_dia:.2f}</td>
        <td style='text-align: center;'><font style='color: {cor_faturamento_dia}'>◙</font></td>
    </tr>

    <tr>
        <td>Diversidade de Produtos</td>
        <td style='text-align: center;'>{quantidade_produtos_dia}</td>
        <td style='text-align: center;'>{meta_qtd_produtos_dia}</td>
        <td style='text-align: center;'><font style='color: {cor_quantidade_dia}'>◙</font></td>
    </tr>

    <tr>
        <td >Ticket Médio</td>
        <td style='text-align: center;'>R$ {ticket_medio_dia:.2f}</td>
        <td style='text-align: center;'>R$ {meta_ticket_medio_dia:.2f}</td>
        <td style='text-align: center;'><font style='color: {cor_ticked_dia}'>◙</font></td>
    </tr>

    </table>

    <br/>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cénario Ano</th>
    </tr>

    <tr>
        <td>Faturamento</td>
        <td style='text-align: center;'>R$ {faturamento_ano:.2f}</td>
        <td style='text-align: center;'>R$ {meta_faturamento_ano:.2f}</td>
        <td style='text-align: center;'><font style='color: {cor_faturamento_ano}'>◙</font></td>
    </tr>

    <tr>
        <td>Diversidade de Produtos</td>
        <td style='text-align: center;'>{quantidade_produtos_ano}</td>
        <td style='text-align: center;'>{meta_qtd_produtos_ano}</td>
        <td style='text-align: center;'><font style='color: {cor_quantidade_ano}'>◙</font></td>
    </tr>

    <tr>
        <td >Ticket Médio</td>
        <td style='text-align: center;'>R$ {ticket_medio_ano:.2f}</td>
        <td style='text-align: center;'>R$ {meta_ticket_medio_ano:.2f}</td>
        <td style='text-align: center;'><font style='color: {cor_ticked_ano}'>◙</font></td>
    </tr>

    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou á disposição.</p>


    <p>Att., Matheus</p>
    """

    #Anexos 
    attachment = Path.cwd() / caminho_backup / loja / f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
    mail.Attachments.Add(str(attachment))

    #Enviar o e-mail
    mail.Send()

    print(f'E-mail da loja: {loja} enviado...')

In [66]:
#Criar ranking de lojas para a diretoria

faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)


nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_Ranking Anual.xlsx'
faturamento_lojas_ano.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))

vendas_dia = vendas.loc[vendas['Data']== dia_indicador, :]
faturamento_lojas_dia =  vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False) 

nome_arquivo = f'{dia_indicador.day}_{dia_indicador.month}_Ranking Dia.xlsx'
faturamento_lojas_dia.to_excel(r'Backup Arquivos Lojas\{}'.format(nome_arquivo))


In [78]:
#Envio de e-mail
outlook = win32.Dispatch('outlook.application')


mail = outlook.CreateItem(0)
mail.To = emails_df.loc[emails_df['Loja'] == 'Diretoria' ,'E-mail'].values[0]
mail.Subject = f'Ranking Dia {dia_indicador.day}/{dia_indicador.month}'


mail.body = f"""
Prezados, Bom dia,

Melhor loja do dia em faturamento: Loja: {faturamento_lojas_dia.index[0]} com Faturamento R$ {faturamento_lojas_dia.iloc[0,0]:.2f}
Pior loja do dia em faturamento: Loja: {faturamento_lojas_dia.index[-1]} com Faturamento R$ {faturamento_lojas_dia.iloc[-1,-1]:.2f}


Melhor loja do ano em faturamento: Loja: {faturamento_lojas_ano.index[0]} com Faturamento R$ {faturamento_lojas_ano.iloc[0,0]:.2f}
Pior loja do ano em faturamento: Loja: {faturamento_lojas_ano.index[-1]} com Faturamento R$ {faturamento_lojas_ano.iloc[-1,-1]:.2f}


Segue em anexo o ranking do ano e do dia de todas as lojas.

Qualquer dúvida estou a disposição.


Att., Matheus
"""

#Anexos 
attachment = Path.cwd() / caminho_backup  / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Anual.xlsx'
mail.Attachments.Add(str(attachment))

attachment = Path.cwd() / caminho_backup  / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Dia.xlsx'
mail.Attachments.Add(str(attachment))

#Enviar o e-mail
mail.Send()
print('E-mail para a diretoria enviado')



E-mail para a diretoria enviado
